In [1]:
import torch
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
datasets_path = Path("/home/codeastra/datasets/")
jan_dset_path =  datasets_path / "fhv_tripdata_2021-01.parquet"
feb_dset_path =  datasets_path / "fhv_tripdata_2021-02.parquet"


In [3]:
jan_dset = pd.read_parquet(jan_dset_path)
feb_dset = pd.read_parquet(feb_dset_path)

In [4]:
print(f"THe number of records in Jan dataset: {len(jan_dset)}")

THe number of records in Jan dataset: 1154112


In [5]:
jan_dset["month"] = 1
feb_dset["month"] = 2
df = pd.concat([jan_dset, feb_dset])

In [6]:
df

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,month
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009,1
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009,1
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013,1
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037,1
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037,1
...,...,...,...,...,...,...,...,...
1037687,B03282,2021-02-28 23:01:16,2021-02-28 23:14:48,NaN,31.0,None,B01717,2
1037688,B03282,2021-02-28 23:36:10,2021-02-28 23:47:38,NaN,169.0,None,B01717,2
1037689,B03285,2021-02-28 23:18:36,2021-02-28 23:43:59,28.0,171.0,None,B03285,2
1037690,B03285,2021-02-28 23:26:34,2021-02-28 23:44:37,16.0,252.0,None,B03285,2


In [7]:
df['duration'] = df.dropOff_datetime - df.pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [8]:
print(f"The mean of a ride duration is: {(df[df['month'] == 1].duration.mean())}")

The mean of a ride duration is: 19.167224093791006


In [9]:
print(f"The number of dropped recrds is: {len(df) - len(df[(df.duration >= 1) & (df.duration <= 60)])}")
df = df[(df.duration >= 1) & (df.duration <= 60)]

The number of dropped recrds is: 91865


In [10]:
df['PUlocationID'] = df['PUlocationID'].fillna(-1)
df['DOlocationID']= df['DOlocationID'].fillna(-1)
print((df['PUlocationID'] == -1).sum())


1775669


/tmp/ipykernel_535/4220689512.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PUlocationID'] = df['PUlocationID'].fillna(-1)
/tmp/ipykernel_535/4220689512.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DOlocationID']= df['DOlocationID'].fillna(-1)


In [11]:
(((df[df['month'] == 1]['PUlocationID'] == -1).sum())/len(df[df['month'] == 1]))*100

83.52732770722618

In [12]:
print(pd.get_dummies(df[df['month'] == 1]['PUlocationID']).shape[1] + pd.get_dummies(df[df['month'] == 1]['DOlocationID']).shape[1])

525


In [13]:
df_train = df[df['month'] == 1]
df_val = df[df['month'] == 2]

In [14]:
df_train['PU_DO'] = df_train['PUlocationID'].astype(str) + '_' + df_train['DOlocationID'].astype(str)
df_val['PU_DO'] = df_val['PUlocationID'].astype(str) + '_' + df_val['DOlocationID'].astype(str)


/tmp/ipykernel_535/3542422261.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['PU_DO'] = df_train['PUlocationID'].astype(str) + '_' + df_train['DOlocationID'].astype(str)
/tmp/ipykernel_535/3542422261.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_val['PU_DO'] = df_val['PUlocationID'].astype(str) + '_' + df_val['DOlocationID'].astype(str)


In [15]:
dv = DictVectorizer()
df_train[['PUlocationID', 'DOlocationID']] = df_train[['PUlocationID', 'DOlocationID']].astype(int)
df_train[['PUlocationID', 'DOlocationID']] = df_train[['PUlocationID', 'DOlocationID']].astype(str)
train_dicts = df_train[['PUlocationID', 'DOlocationID']].to_dict(orient='reports')
print(len(train_dicts))
X_train = dv.fit_transform(train_dicts)

/tmp/ipykernel_535/2643069311.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[['PUlocationID', 'DOlocationID']] = df_train[['PUlocationID', 'DOlocationID']].astype(int)
/tmp/ipykernel_535/2643069311.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[['PUlocationID', 'DOlocationID']] = df_train[['PUlocationID', 'DOlocationID']].astype(str)
/tmp/ipykernel_535/2643069311.py:4: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 

1109826


In [16]:
X_train

<1109826x525 sparse matrix of type '<class 'numpy.float64'>'
	with 2219652 stored elements in Compressed Sparse Row format>

In [17]:
y_train = df_train['duration'].values


In [18]:
print(y_train.shape)

(1109826,)


In [19]:
print(X_train.shape)
print(y_train.shape)


lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)


(1109826, 525)
(1109826,)


10.528519427064724

In [21]:
y_val = df_val['duration'].values


dv = DictVectorizer()
df_val[['PUlocationID', 'DOlocationID']] = df_val[['PUlocationID', 'DOlocationID']].astype(int)
df_val[['PUlocationID', 'DOlocationID']] = df_val[['PUlocationID', 'DOlocationID']].astype(str)
train_dicts = df_val[['PUlocationID', 'DOlocationID']].to_dict(orient='reports')
print(len(train_dicts))
'''X_val = dv.fit_transform(train_dicts)


y_pred = lr.predict(X_val)
print(mean_squared_error(y_val, y_pred, squared=False))'''


/tmp/ipykernel_535/2552844692.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_val[['PUlocationID', 'DOlocationID']] = df_val[['PUlocationID', 'DOlocationID']].astype(int)
/tmp/ipykernel_535/2552844692.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_val[['PUlocationID', 'DOlocationID']] = df_val[['PUlocationID', 'DOlocationID']].astype(str)
/tmp/ipykernel_535/2552844692.py:7: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records

: 

: 